In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

In [2]:
from pathlib import Path

data_dir = Path('data')
train_data = pd.read_csv(data_dir / 'train.csv')
test_data = pd.read_csv(data_dir / 'test.csv')

train_data.shape

(42000, 785)

In [6]:
from tensorflow.random import set_seed 
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.utils import to_categorical

def prepage_input_shape(data: pd.DataFrame):
    return data.values.reshape(-1, 28, 28)


model = keras.Sequential()
model.add(Conv2D(
              1, 
              kernel_size=4,
              activation='relu',
              input_shape=(28, 28, 1)
            )
         )
model.add(Conv2D(
              1, 
              kernel_size=4,
              activation='sigmoid',
              input_shape=(28, 28, 1)
            )
         )
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_hinge', metrics=['accuracy'])

y = to_categorical(train_data.label.values) # one-hot encode target 
X = prepage_input_shape(train_data.drop('label', axis=1))

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.005,
    patience=8,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [ ]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    validation_data=[X_test, y_test],
                    callbacks=[early_stopping])

In [28]:
# train on all data
early_stopping.monitor = 'accuracy'
history = model.fit(X,
                    y,
                    epochs=50,
                    callbacks=[early_stopping])

Epoch 1/50
1313/1313 [==============================] - 6s 5ms/step - loss: 0.1207 - accuracy: 0.9457
Epoch 2/50
1313/1313 [==============================] - 7s 5ms/step - loss: 0.1177 - accuracy: 0.9469
Epoch 3/50
1313/1313 [==============================] - 7s 5ms/step - loss: 0.1157 - accuracy: 0.9477
Epoch 4/50
1313/1313 [==============================] - 6s 5ms/step - loss: 0.1139 - accuracy: 0.9492
Epoch 5/50
1313/1313 [==============================] - 6s 5ms/step - loss: 0.1125 - accuracy: 0.9492
Epoch 6/50
1313/1313 [==============================] - 7s 6ms/step - loss: 0.1107 - accuracy: 0.9505
Epoch 7/50
1313/1313 [==============================] - 8s 6ms/step - loss: 0.1091 - accuracy: 0.9507
Epoch 8/50
1313/1313 [==============================] - 8s 6ms/step - loss: 0.1081 - accuracy: 0.9518
Epoch 9/50
1313/1313 [==============================] - 7s 6ms/step - loss: 0.1071 - accuracy: 0.9518
Epoch 10/50
1313/1313 [==============================] - 7s 6ms/step - loss: 0.105

In [15]:
# naive data augmentation - duplicate wrong predication samples

aug_candidate_X = X
aug_candidate_y = y

all_preds = [x.argmax() for x in model.predict(aug_candidate_X)]

hits_and_misses = zip(X_train, 
                      [p == v.argmax() for (p, v) in zip(all_preds, aug_candidate_y)], 
                      aug_candidate_y)

misses = [(x, _y) for (x, hit, _y) in hits_and_misses if hit == False]
misses_X = [item[0] for item in misses]
misses_y = [item[1] for item in misses]

# X5 augmentation of problematic data
for _ in range(2): 
    misses_X = np.concatenate([misses_X, misses_X])
    misses_y = np.concatenate([misses_y, misses_y])
    
augmented_X = np.concatenate([X, misses_X])
augmented_y = np.concatenate([y, misses_y])
augmented_y

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [18]:
# train on augmented data
early_stopping.monitor = 'accuracy'
history = model.fit(augmented_X,
                    augmented_y,
                    epochs=30,)
#                     validation_data=[X_test, y_test],
#                     callbacks=[early_stopping])

Epoch 1/30
1602/1602 [==============================] - 9s 6ms/step - loss: 0.4405 - accuracy: 0.7818
Epoch 2/30
1602/1602 [==============================] - 7s 4ms/step - loss: 0.4386 - accuracy: 0.7830
Epoch 3/30
1602/1602 [==============================] - 8s 5ms/step - loss: 0.4375 - accuracy: 0.7838
Epoch 4/30
1602/1602 [==============================] - 12s 7ms/step - loss: 0.4384 - accuracy: 0.7823
Epoch 5/30
1602/1602 [==============================] - 10s 6ms/step - loss: 0.4370 - accuracy: 0.7834
Epoch 6/30
1602/1602 [==============================] - 10s 6ms/step - loss: 0.4354 - accuracy: 0.7845
Epoch 7/30
1602/1602 [==============================] - 11s 7ms/step - loss: 0.4348 - accuracy: 0.7846
Epoch 8/30
1602/1602 [==============================] - 11s 7ms/step - loss: 0.4338 - accuracy: 0.7845
Epoch 9/30
1602/1602 [==============================] - 15s 9ms/step - loss: 0.4321 - accuracy: 0.7858
Epoch 10/30
1602/1602 [==============================] - 13s 8ms/step - loss

In [30]:
# val_accuracy: 0.9214
from sklearn.metrics import accuracy_score



import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(10,5))
plt.show()
accuracy_score([pred.argmax() for pred in model.predict(X)], [y_.argmax() for y_ in y])

NameError: name 'history' is not defined

In [30]:
import csv
from typing import List

# Submittion Format:
# ImageId,Label
# 1,3
# 2,7
# 3,8 
# (27997 more lines)

def prepare_preds_for_submission(predictions: List[int]):
    fields = ['ImageId', 'Label']
    # data rows of csv file
    predictions = [x.argmax() for x in predictions]
    rows = enumerate(predictions, start=1)

    with open('submission.csv', 'w') as f:
        write = csv.writer(f)
        write.writerow(fields)
        write.writerows(rows)

test_predictions = model.predict(prepage_input_shape(test_data))
prepare_preds_for_submission(test_predictions)
pd.read_csv('submission.csv')
# [x.argmax() for x in test_predictions]

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [359]:
size = 15

targets = [x.argmax() for x in y_test[:size]]
preds = [x.argmax() for x in model.predict(X_test[:size])]

# targets = [x.argmax() for x in y_train[:size]]
# preds = [x.argmax() for x in model.predict(X_train[:size])]

for _ in zip(targets, preds):
    print(_)
targets == preds

(8, 8)
(1, 1)
(9, 9)
(9, 9)
(8, 8)
(6, 6)
(2, 2)
(2, 7)
(7, 7)
(1, 1)
(6, 6)
(3, 3)
(1, 8)
(2, 2)
(7, 7)


False

In [ ]:
from tensorflow import keras
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def add_channels(img):
    return np.array([img, img, img]) # pseudo 3-channels


def prepage_input_shape_for_inception(data: pd.DataFrame):
    data = np.array(
        [add_channels(np.lib.pad(x.copy(), 120, constant_values=0))
         for x in data.drop('label', axis=1).values]
    ) # add padding to support 32X32 (32^2=1024) original size is 28^2=784 plus 120*2 (padding from both sides)
    return data.reshape(-1, 32, 32, 3)


vgg16 = VGG16(input_shape=(32, 32, 3), include_top=False, classes=10)
model = keras.Sequential()
model.add(vgg16)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation='softmax'))

opt = keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer="adam", loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

X_inception = prepage_input_shape_for_inception(train_data)
y = to_categorical(train_data.label.values) # one-hot encode target 
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


history = model.fit(X_inception, y)

   4/1313 [..............................] - ETA: 15:57 - loss: 20.6020 - accuracy: 0.1016

In [39]:
from sklearn.metrics import accuracy_score

size = 3000

def to_classes(predictions: np.ndarray) -> int:
    return [p.argmax() for p in predictions]

preds = model.predict(prepage_input_shape_for_inception(train_data.iloc[:size]))
preds = to_classes(preds)
accuracy_score(preds, to_classes(y[:size]))

0.114